### 데이터 분석과 전처리

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train= pd.read_csv("quora-question-pairs/train.csv")
test= pd.read_csv("quora-question-pairs/test.csv")

In [3]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
test.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [5]:
len(train)

404290

In [6]:
len(test)

2345796

### 데이터 전처리

In [7]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [8]:
train_pos= train[train['is_duplicate']==1]
train_neg= train[train['is_duplicate']==0]

In [9]:
## 라벨 개수의 균형을 맞추는 작업
class_difference= len(train_neg)-len(train_pos)
sample_frac= 1- (class_difference/len(train_neg))
train_neg= train_neg.sample(frac=sample_frac)

In [10]:
print(len(train_pos))

149263


In [11]:
print(len(train_neg))

149263


In [12]:
train= pd.concat([train_pos,train_neg])

In [13]:
import re

## 리스트 형태로 변환 후, 불필요한 물음표, 마침표 등 제거
questions1= [str(s) for s in train['question1']]
questions2= [str(s) for s in train['question2']]

filtered_questions1= list()
filtered_questions2= list()

for q in questions1:
    filtered_questions1.append(re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', q).lower())
    
for q in questions2:
    filtered_questions2.append(re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', q).lower())

In [14]:
filtered_questions1

['astrology i am a capricorn sun cap moon and cap risingwhat does that say about me',
 'how can i be a good geologist',
 'how do i read and find my youtube comments',
 'what can make physics easy to learn',
 'what was your first sexual experience like',
 'what would a trump presidency mean for current international master’s students on an f1 visa',
 'what does manipulation mean',
 'why are so many quora users posting questions that are readily answered on google',
 'why do rockets look white',
 'how should i prepare for ca final law',
 'what are some special cares for someone with a nose that gets stuffy during the night',
 'what game of thrones villain would be the most likely to give you mercy',
 'how do we prepare for upsc',
 'what are some examples of products that can be make from crude oil',
 'how do i make friends',
 'is career launcher good for rbi grade b preparation',
 'will a blu ray play on a regular dvd player if so how',
 'what is the bestmost memorable thing youve ever e

In [15]:
## 문서 토크나이징
tokenizer= Tokenizer()
tokenizer.fit_on_texts(filtered_questions1 + filtered_questions2)

In [16]:
## 단어를 인덱스로 변환
questions1_sequence= tokenizer.texts_to_sequences(filtered_questions1)
questions2_sequence= tokenizer.texts_to_sequences(filtered_questions2)

In [17]:
questions1_sequence

[[2125, 5, 79, 7, 9341, 922, 5269, 756, 12, 5269, 47993, 21, 28, 246, 46, 64],
 [4, 13, 5, 23, 7, 41, 26066],
 [4, 9, 5, 216, 12, 88, 18, 272, 1731],
 [2, 13, 53, 619, 584, 6, 66],
 [2, 61, 32, 100, 1417, 294, 40],
 [2, 42, 7, 75, 923, 114, 14, 489, 514, 8232, 278, 20, 33, 3184, 538],
 [2, 21, 11948, 114],
 [16, 11, 62, 81, 47, 813, 3011, 105, 28, 11, 9776, 1030, 20, 134],
 [16, 9, 8998, 273, 355],
 [4, 30, 5, 181, 14, 542, 976, 521],
 [2, 11, 25, 1007, 7555, 14, 87, 31, 7, 1916, 28, 1170, 11949, 225, 1, 670],
 [2, 298, 10, 1185, 5009, 42, 23, 1, 49, 799, 6, 239, 15, 12289],
 [4, 9, 54, 181, 14, 1064],
 [2, 11, 25, 190, 10, 557, 28, 13, 23, 53, 34, 6776, 821],
 [4, 9, 5, 53, 266],
 [3, 251, 9342, 41, 14, 1419, 1560, 971, 510],
 [36, 7, 18659, 5171, 315, 20, 7, 1598, 4635, 1422, 24, 62, 4],
 [2, 3, 1, 23470, 5637, 152, 375, 78, 3689, 12, 16],
 [5,
  61,
  1666,
  5458,
  321,
  342,
  5,
  227,
  1041,
  18,
  342,
  247,
  12,
  168,
  5798,
  1,
  1017,
  307,
  3,
  183,
  1198,
  18

In [18]:
## 길이가 긴 질문을 제거하는 패딩 과정을 진행 (짧은 부분은 0으로 대체)
max_len= 31

train_q1_data= pad_sequences(questions1_sequence, maxlen= max_len, padding='post')
train_q2_data= pad_sequences(questions2_sequence, maxlen= max_len, padding='post')

In [19]:
word_vocab= {}
word_vocab= tokenizer.word_index
word_vocab["<PAD>"]=0

In [20]:
train_labels= np.array(train['is_duplicate'], dtype=int)

In [21]:
print('shape of q1 data: {}'.format(train_q1_data.shape))
print('shape of q2 data: {}'.format(train_q2_data.shape))
print('Words in index: {}'.format(len(word_vocab)))

shape of q1 data: (298526, 31)
shape of q2 data: (298526, 31)
Words in index: 87353


In [22]:
data_configs={}
data_configs['vocab']= word_vocab
data_configs['vocab_size']= len(word_vocab)

In [23]:
data_configs

{'vocab': {'the': 1,
  'what': 2,
  'is': 3,
  'how': 4,
  'i': 5,
  'to': 6,
  'a': 7,
  'in': 8,
  'do': 9,
  'of': 10,
  'are': 11,
  'and': 12,
  'can': 13,
  'for': 14,
  'you': 15,
  'why': 16,
  'best': 17,
  'my': 18,
  'it': 19,
  'on': 20,
  'does': 21,
  'which': 22,
  'be': 23,
  'if': 24,
  'some': 25,
  'or': 26,
  'get': 27,
  'that': 28,
  'have': 29,
  'should': 30,
  'with': 31,
  'your': 32,
  'an': 33,
  'from': 34,
  'india': 35,
  'will': 36,
  'people': 37,
  'who': 38,
  'when': 39,
  'like': 40,
  'good': 41,
  'would': 42,
  'there': 43,
  'at': 44,
  'as': 45,
  'about': 46,
  'quora': 47,
  'between': 48,
  'most': 49,
  'way': 50,
  'not': 51,
  'one': 52,
  'make': 53,
  'we': 54,
  'did': 55,
  'by': 56,
  'life': 57,
  'where': 58,
  'any': 59,
  'money': 60,
  'was': 61,
  'so': 62,
  'after': 63,
  'me': 64,
  'time': 65,
  'learn': 66,
  'they': 67,
  'this': 68,
  'know': 69,
  'difference': 70,
  'whats': 71,
  'has': 72,
  'new': 73,
  'think': 74,

In [24]:
## 테스트셋도 동일하게 적용
## 리스트 형태로 변환 후, 불필요한 물음표, 마침표 등 제거
questions1= [str(s) for s in test['question1']]
questions2= [str(s) for s in test['question2']]

filtered_questions1= list()
filtered_questions2= list()

for q in questions1:
    filtered_questions1.append(re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', q).lower())
    
for q in questions2:
    filtered_questions2.append(re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', q).lower())
    
import re

## 리스트 형태로 변환 후, 불필요한 물음표, 마침표 등 제거
questions1= [str(s) for s in test['question1']]
questions2= [str(s) for s in test['question2']]

filtered_questions1= list()
filtered_questions2= list()

for q in questions1:
    filtered_questions1.append(re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', q).lower())
    
for q in questions2:
    filtered_questions2.append(re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', q).lower())

## 문서 토크나이징
tokenizer= Tokenizer()
tokenizer.fit_on_texts(filtered_questions1 + filtered_questions2)

## 단어를 인덱스로 변환
questions1_sequence= tokenizer.texts_to_sequences(filtered_questions1)
questions2_sequence= tokenizer.texts_to_sequences(filtered_questions2)
    
## 길이가 긴 질문을 제거하는 패딩 과정을 진행 (짧은 부분은 0으로 대체)
max_len= 31

test_q1_data= pad_sequences(questions1_sequence, maxlen= max_len, padding='post')
test_q2_data= pad_sequences(questions2_sequence, maxlen= max_len, padding='post')

In [25]:
len(test_q1_data)

2345796

In [26]:
test_id= np.array(test['test_id'])

In [27]:
len(test_id)

2345796

### 데이터 저장

In [28]:
np.save(open('quora-question-pairs/q1_train.npy','wb'),train_q1_data)
np.save(open('quora-question-pairs/q2_train.npy','wb'),train_q2_data)
np.save(open('quora-question-pairs/label_train.npy','wb'),train_labels)

import json

json.dump(data_configs, open('quora-question-pairs/data_configs.npy','w'))

In [29]:
np.save(open('quora-question-pairs/q1_test.npy','wb'),test_q1_data)
np.save(open('quora-question-pairs/q2_test.npy','wb'),test_q2_data)
np.save(open('quora-question-pairs/test_id.npy','wb'),test_id)